In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [58]:
def load_data(path_sy,path_an):
    synonym = pd.read_csv(path_sy, sep=' ', header=None)
    antonym = pd.read_csv(path_an, sep=' ', header=None)

    synonym['label'] = 1
    antonym['label'] = 0

    data = pd.concat([synonym, antonym], ignore_index=True)
    data.columns = ['word1', 'word2', 'label']

    missing_data = data[data['word2'].isna() | (data['word2'] == '')]
    print("Dữ liệu miss được loại bỏ:")
    print(missing_data)

    data = data[data['word2'].notna() & (data['word2'] != '')]
    

    data['pair'] = data['word1'] + ' ' + data['word2']
    X = data['pair']
    y = data['label']

    return X, y


In [59]:
X, y = load_data('an-sy/Synonym_vietnamese.txt', 'an-sy/Antonym_vietnamese.txt')

Dữ liệu miss được loại bỏ:
           word1 word2  label
562   bằng_chứng   NaN      1
2941  dài_ngoằng   NaN      1
3097    dầu_nhớt   NaN      1
3195   diễn_dịch   NaN      1


In [74]:
X

0            a_dua a_tòng
1           a_dua vào_hùa
2          a_ma_tơ tài_tử
3          a_tòng vào_hùa
4               à_ơi ạ_ơi
               ...       
13557    bẩn_thỉu sạch_sẽ
13558            bận rảnh
13559       bận rảnh_rang
13560        bận rảnh_rỗi
13561             bận rỗi
Name: pair, Length: 13558, dtype: object

In [71]:
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

    vectorizer = CountVectorizer()
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    model = LogisticRegression()
    model.fit(X_train_vec, y_train)

    y_pred = model.predict(X_test_vec)

    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Classification report:')
    print(classification_report(y_test, y_pred))
    print('Confusion matrix:')
    print(confusion_matrix(y_test, y_pred))
    print()

    return model, vectorizer

In [73]:
model, vectorizer = train_model(X, y)

def predict(model, vectorizer, word1, word2):
    pair = word1 + ' ' + word2
    pair_vec = vectorizer.transform([pair])

    pred = model.predict(pair_vec)[0]

    if pred == 1:
        return '{word1} và {word2} là từ đồng nghĩa'.format(word1=word1, word2=word2)
    else:
        return '{word1} và {word2} là từ trái nghĩa'.format(word1=word1, word2=word2)
    
print(predict(model, vectorizer, 'đi', 'đứng'))

Accuracy: 0.8978613569321534
Classification report:
              precision    recall  f1-score   support

           0       0.83      0.39      0.53       401
           1       0.90      0.99      0.94      2311

    accuracy                           0.90      2712
   macro avg       0.87      0.69      0.74      2712
weighted avg       0.89      0.90      0.88      2712

Confusion matrix:
[[ 155  246]
 [  31 2280]]

đi và đứng là từ đồng nghĩa
